<a href="https://colab.research.google.com/github/Bnrobertson/Rainfall_Project/blob/Neural_network/Rainfall_Project_LR_Model_Gcolab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import our dependencies

import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import drive
from sqlalchemy import create_engine
from pathlib import Path
import os
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
!pip install keras-tuner
import keras_tuner as kt

!pip install pandas psycopg2-binary SQLAlchemy

spark_version = 'spark-3.5.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 654.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 5.3 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,377 kB]
Get:11 http://security.ubuntu.com/ubuntu jam

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
Rainfall_df = pd.read_csv("/content/drive/MyDrive/Rainfall_Project_folder/Rainfall_OG.csv")
Humidity_df = pd.read_csv("/content/drive/MyDrive/Rainfall_Project_folder/Rainfall_YB2.csv")

In [6]:
Merged_df = pd.merge(Rainfall_df, Humidity_df, on="Date")
Merged_df.head()

,Date,TempAvgºF,TempMinºF,TempMaxºF,RainTotal,WindSpeed,Pressure,Raining,HumidityMax,HumidityAvg,HumidityMin
0,2000-01-01,50.00,33.08,66.92,0.000000,2.920444,30.247579,False,100.0,73.6,40.0
1,2000-01-02,57.02,42.98,69.98,0.000000,7.580726,30.223955,False,86.0,73.2,56.0
2,2000-01-03,64.94,55.04,73.94,0.000000,10.314759,30.150130,False,100.0,81.9,51.0
3,2000-01-04,62.06,51.08,73.04,0.314961,15.907098,29.884360,True,100.0,84.2,59.0
4,2000-01-05,39.02,26.96,51.08,0.000000,5.840887,30.247579,False,92.0,57.5,37.0


In [7]:
# Drop date column
Current_Merged_df = Merged_df.drop(columns=["Date", "RainTotal", "TempAvgºF", "HumidityMax", "HumidityMin"])
Current_Merged_df.head()

,TempMinºF,TempMaxºF,WindSpeed,Pressure,Raining,HumidityAvg
0,33.08,66.92,2.920444,30.247579,False,73.6
1,42.98,69.98,7.580726,30.223955,False,73.2
2,55.04,73.94,10.314759,30.150130,False,81.9
3,51.08,73.04,15.907098,29.884360,True,84.2
4,26.96,51.08,5.840887,30.247579,False,57.5


In [8]:
# Split our preprocessed data into our features and target arrays

y = Current_Merged_df["Raining"].values
X = Current_Merged_df.drop(columns=["Raining"]).values

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [11]:
# Train the data
classifier.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [12]:
# Score the model
print(f"Training Data Score: {classifier.score(X_train, y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, y_test)}")

Training Data Score: 0.8352195563603441
Testing Data Score: 0.8470588235294118


In [13]:
# Predict outcomes for test data set
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})


,Prediction,Actual
0,False,False
1,False,False
2,True,True
3,False,False
4,False,False
...,...,...
2205,False,False
2206,False,False
2207,False,False
2208,False,False


In [14]:
from sklearn.metrics import accuracy_score

# Display the accuracy score for the test dataset.
accuracy_score(y_test, predictions)

0.8470588235294118

In [15]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[1541,  127],
       [ 211,  331]])

In [17]:
with open("rainfall_model.pkl", "wb") as f:
    pickle.dump(classifier, f)